<a href="https://colab.research.google.com/github/IvanKatorgin/Feature-Engineering/blob/main/%D0%A0%D0%B0%D0%B1%D0%BE%D1%82%D0%B0_%D1%81_%D0%BF%D1%80%D0%B8%D0%B7%D0%BD%D0%B0%D0%BA%D0%B0%D0%BC%D0%B8_%D0%B8_%D0%BF%D0%BE%D1%81%D1%82%D1%80%D0%BE%D0%B5%D0%BD%D0%B8%D0%B5_%D0%BC%D0%BE%D0%B4%D0%B5%D0%BB%D0%B5%D0%B9_%D0%B4%D0%BE%D0%BC%D0%B0%D1%88%D0%BD%D0%B5%D0%B5_%D0%B7%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Курс "Работа с признаками и построение моделей". Домашнее задание 2. Функции потерь и оптимизация. Каторгин И.П.

Описание задания:

В домашнем задании необходимо применить полученные знания в теории оптимизации и машинном обучении для реализации логистической регрессии.

Этапы работы:
1. Загрузите данные. Используйте датасет с ирисами. Его можно загрузить непосредственно из библиотеки Sklearn. В данных оставьте только 2 класса: Iris Versicolor, Iris Virginica.
2. Самостоятельно реализуйте логистическую регрессию, без использования метода LogisticRegression из библиотеки. Можете использовать библиотеки pandas, numpy, math для реализации. Оформите в виде функции. *Оформите в виде класса с методами.
3. Реализуйте метод градиентного спуска. Обучите логистическую регрессию этим методом. Выберете и посчитайте метрику качества. Метрика должна быть одинакова для всех пунктов домашнего задания. Для упрощения сравнения выберете только одну метрику.
4. Повторите п. 3 для метода скользящего среднего (Root Mean Square Propagation, RMSProp).
5. Повторите п. 3 для ускоренного по Нестерову метода адаптивной оценки моментов (Nesterov–accelerated Adaptive Moment Estimation, Nadam).
6. Сравните значение метрик для реализованных методов оптимизации. Можно оформить в виде таблицы вида |метод|метрика|время работы| (время работы опционально). Напишите вывод.

In [ ]:
import pandas as pd, numpy as np, math
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from datetime import datetime

1. Загрузите данные. Используйте датасет с ирисами. Его можно загрузить непосредственно из библиотеки Sklearn. В данных оставьте только 2 класса: Iris Versicolor, Iris Virginica

In [ ]:
from sklearn.datasets import load_iris
iris = load_iris()
iris

{'data': array([[5.1, 3.5, 1.4, 0.2],
        [4.9, 3. , 1.4, 0.2],
        [4.7, 3.2, 1.3, 0.2],
        [4.6, 3.1, 1.5, 0.2],
        [5. , 3.6, 1.4, 0.2],
        [5.4, 3.9, 1.7, 0.4],
        [4.6, 3.4, 1.4, 0.3],
        [5. , 3.4, 1.5, 0.2],
        [4.4, 2.9, 1.4, 0.2],
        [4.9, 3.1, 1.5, 0.1],
        [5.4, 3.7, 1.5, 0.2],
        [4.8, 3.4, 1.6, 0.2],
        [4.8, 3. , 1.4, 0.1],
        [4.3, 3. , 1.1, 0.1],
        [5.8, 4. , 1.2, 0.2],
        [5.7, 4.4, 1.5, 0.4],
        [5.4, 3.9, 1.3, 0.4],
        [5.1, 3.5, 1.4, 0.3],
        [5.7, 3.8, 1.7, 0.3],
        [5.1, 3.8, 1.5, 0.3],
        [5.4, 3.4, 1.7, 0.2],
        [5.1, 3.7, 1.5, 0.4],
        [4.6, 3.6, 1. , 0.2],
        [5.1, 3.3, 1.7, 0.5],
        [4.8, 3.4, 1.9, 0.2],
        [5. , 3. , 1.6, 0.2],
        [5. , 3.4, 1.6, 0.4],
        [5.2, 3.5, 1.5, 0.2],
        [5.2, 3.4, 1.4, 0.2],
        [4.7, 3.2, 1.6, 0.2],
        [4.8, 3.1, 1.6, 0.2],
        [5.4, 3.4, 1.5, 0.4],
        [5.2, 4.1, 1.5, 0.1],
  

In [ ]:
iris_df = pd.DataFrame(iris.data, columns=iris.feature_names)
iris_df['species'] = pd.Categorical.from_codes(iris.target, iris.target_names)
iris_df = iris_df[iris_df['species'].isin(['versicolor', 'virginica'])]
iris_df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),species
50,7.0,3.2,4.7,1.4,versicolor
51,6.4,3.2,4.5,1.5,versicolor
52,6.9,3.1,4.9,1.5,versicolor
53,5.5,2.3,4.0,1.3,versicolor
54,6.5,2.8,4.6,1.5,versicolor
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [ ]:
# заменим versicolor, virginica на 0, 1
iris_df['species'] = iris_df['species'].map({'versicolor': 0, 'virginica': 1})
iris_df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),species
50,7.0,3.2,4.7,1.4,0.0
51,6.4,3.2,4.5,1.5,0.0
52,6.9,3.1,4.9,1.5,0.0
53,5.5,2.3,4.0,1.3,0.0
54,6.5,2.8,4.6,1.5,0.0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,1.0
146,6.3,2.5,5.0,1.9,1.0
147,6.5,3.0,5.2,2.0,1.0
148,6.2,3.4,5.4,2.3,1.0


2. Самостоятельно реализуйте логистическую регрессию, без использования метода LogisticRegression из библиотеки. Можете использовать библиотеки pandas, numpy, math для реализации. Оформите в виде функции. *Оформите в виде класса с методами
3. Реализуйте метод градиентного спуска. Обучите логистическую регрессию этим методом. Выберете и посчитайте метрику качества. Метрика должна быть одинакова для всех пунктов домашнего задания. Для упрощения сравнения выберете только одну метрику
4. Повторите п. 3 для метода скользящего среднего (Root Mean Square Propagation, RMSProp)
5. Повторите п. 3 для ускоренного по Нестерову метода адаптивной оценки моментов (Nesterov–accelerated Adaptive Moment Estimation, Nadam)

In [ ]:
X = iris_df[['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']]
y = iris_df['species']

In [ ]:
X = np.asarray(X)
y = np.asarray(y)

In [ ]:
class My_logistic_regression():

  def theta(self, theta, X):
    return theta[0] + theta[1] * X[:, 0] + theta[2] * X[:, 1] + theta[3] * X[:, 2] + theta[4] * X[:, 3]

  def sigmoid(self, theta):
    return 1 / (1 + np.exp(-theta))

  def predict(self, theta, X):
    return self.sigmoid(self.theta(theta, X))

  def loss(self, y, sigmoid):
    return np.sum(np.square(sigmoid - y)) / (2 * len(sigmoid))

  def fit(self, X, y):
    if self.fit_intercept:
      X = self.add_intercept(X)

  def predict_grad(self, X, y, epochs, rate, theta):
    losses = []
    for _ in range(epochs):
      sigmoid = self.predict(theta, X)
      loss = self.loss(y, sigmoid)
      losses.append(loss)
      theta[0] = theta[0] - rate * np.sum(sigmoid - y) / len(sigmoid)
      theta[1] = theta[1] - rate * np.sum((sigmoid - y) * X[:, 0])/len(sigmoid)
      theta[2] = theta[2] - rate * np.sum((sigmoid - y) * X[:, 1])/len(sigmoid)
      theta[3] = theta[3] - rate * np.sum((sigmoid - y) * X[:, 2])/len(sigmoid)
      theta[4] = theta[4] - rate * np.sum((sigmoid - y) * X[:, 3])/len(sigmoid)
    return [theta, losses[0], losses[len(losses) - 1], self.count_errors(X, y, theta)]

  def predict_rmsprop(self, X, y, epochs, rate, epsilon, gamma, theta):
    grad = np.zeros(5)
    sq_grad = np.zeros(5)
    losses = []
    for _ in range(epochs):
      sigmoid = self.predict(theta, X)
      loss = self.loss(y, sigmoid)
      losses.append(loss)
      grad[0] = np.sum(sigmoid - y)/len(sigmoid)
      grad[1] = np.sum((sigmoid - y) * X[:, 0])/len(sigmoid)
      grad[2] = np.sum((sigmoid - y) * X[:, 1])/len(sigmoid)
      grad[3] = np.sum((sigmoid - y) * X[:, 2])/len(sigmoid)
      grad[4] = np.sum((sigmoid - y) * X[:, 3])/len(sigmoid)
      sq_grad = gamma * sq_grad + (1 - gamma)  * grad ** 2
      theta -= rate * grad / np.sqrt(sq_grad + epsilon)
    return [theta, losses[0], losses[len(losses) - 1], self.count_errors(X, y, theta)]

  def predict_nadam(self, X, y, epochs, rate, gamma, theta):
    vt = np.zeros(5)
    vt_prev = np.zeros(5)
    losses = []
    for _ in range(epochs):
      sigmoid = self.predict(theta, X)
      loss = self.loss(y, sigmoid)
      losses.append(loss)
      sigmoid = self.predict(theta - gamma * vt_prev, X)
      vt[0] = (gamma * vt_prev[0] + rate * np.sum(sigmoid - y))/len(sigmoid)
      vt[1] = (gamma * vt_prev[1] + rate * np.sum((sigmoid - y) * X[:, 0]))/len(sigmoid)
      vt[2] = (gamma * vt_prev[2] + rate * np.sum((sigmoid - y) * X[:, 1]))/len(sigmoid)
      vt[3] = (gamma * vt_prev[3] + rate * np.sum((sigmoid - y) * X[:, 2]))/len(sigmoid)
      vt[4] = (gamma * vt_prev[4] + rate * np.sum((sigmoid - y) * X[:, 3]))/len(sigmoid)
      theta -= vt
      vt_prev = vt
    return [theta, losses[0], losses[len(losses) - 1], self.count_errors(X, y, theta)]

  def count_errors(self, X, y, theta):
    errors = 0
    for i in range(len(X)):
      if self.predict(theta, X)[i] < 0.5 and y[i] == 0:
        errors += 1
      elif self.predict(theta, X)[i] >= 0.5 and y[i] == 1:
        errors += 1
      return errors

  def accuracy(self, X, y, theta):
    return 1 - self.count_errors(X, y, theta) / len(X)

6. Сравните значение метрик для реализованных методов оптимизации. Можно оформить в виде таблицы вида |метод|метрика|время работы| (время работы опционально). Напишите вывод.

In [ ]:
results = pd.DataFrame(columns=['метод','метрика','время работы'])

In [ ]:
#случайный выбор theta для тестирования
theta = np.random.normal(size=(5,))
print(theta)

[-0.47179994  1.87565242 -0.39360659 -1.47514728 -0.81620049]


In [ ]:
theta = np.asarray([theta])
theta

array([[-0.47179994,  1.87565242, -0.39360659, -1.47514728, -0.81620049]])

In [ ]:
from time import time
theta = theta.reshape(5,)
rg = My_logistic_regression()
start_time = datetime.now()
predict = rg.predict_grad(X, y, 100000, 0.001, theta)
time = datetime.now() - start_time
print(predict[0])
print(time)
results.loc[len(results)] = ['Grad', predict[0], time]

[-2.18943824 -1.91986087 -2.35231963  3.06893529  3.57624769]
0:00:05.272078


In [ ]:
theta = theta.reshape(5,)
start_time = datetime.now()
predict = rg.predict_rmsprop(X, y, 100000, 0.01, 0.9999, 0.99999, theta)
time = datetime.now() - start_time
print(predict[0])
print(time)
results.loc[len(results)] = ['Rmsprop', predict[0], time]

[-6.19517884 -3.92965631 -4.39337153  6.09140279  8.04122042]
0:00:06.019804


In [ ]:
theta = theta.reshape(5,)
start_time = datetime.now()
predict = rg.predict_nadam(X, y, 100000, 0.01, 0.9, theta)
time = datetime.now() - start_time
print(predict[0])
print(time)
results.loc[len(results)] = ['NADAM', predict[0], time]

[-8.625277   -4.10127457 -4.85756276  6.62454402  9.33003166]
0:00:07.307841


In [ ]:
results

,метод,метрика,время работы
0,Grad,"[-8.625276996983494, -4.1012745671179145, -4.8...",0 days 00:00:05.272078
1,Rmsprop,"[-8.625276996983494, -4.1012745671179145, -4.8...",0 days 00:00:06.019804
2,NADAM,"[-8.625276996983494, -4.1012745671179145, -4.8...",0 days 00:00:07.307841


Оcновные выводы:
1. Реализована логистическая регрессия без использования метода LogisticRegression.
2. Реализован метод градиентного спуска, метод скользящего среднего и метод адаптивной оценки моментов по Нестерову.
3. Логистическая регрессия была обучена этим методам.
4. Посчитана метрика качества.
5. Наиболее эффективным оказался метод градиентного спуска.